# Preprocessing

This notebook will give you a taste of what `scikit-learn` provides for preprocessing data.

## Data used
We will be using the planets data and red wine data:

### Data License for Planet Data
Copyright (C) 2012 Hanno Rein

Permission is hereby granted, free of charge, to any person obtaining a copy of this database and associated scripts (the "Database"), to deal in the Database without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Database, and to permit persons to whom the Database is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Database. A reference to the Database shall be included in all scientific publications that make use of the Database.

THE DATABASE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE DATABASE OR THE USE OR OTHER DEALINGS IN THE DATABASE.

### Citations for Red Wine Data
P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis. 
Modeling wine preferences by data mining from physicochemical properties.
In Decision Support Systems, Elsevier, 47(4):547-553. ISSN: 0167-9236.

Available at:
- [@Elsevier](http://dx.doi.org/10.1016/j.dss.2009.05.016)
- [Pre-press (pdf)](http://www3.dsi.uminho.pt/pcortez/winequality09.pdf)
- [bib](http://www3.dsi.uminho.pt/pcortez/dss09.bib)

Dua, D. and Karra Taniskidou, E. (2017). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml/index.php](http://archive.ics.uci.edu/ml/index.php). Irvine, CA: University of California, School of Information and Computer Science.

## Setup

In [ ]:
import numpy as np
import pandas as pd

planets = pd.read_csv('data/planets.csv')
red_wine = pd.read_csv('data/winequality-red.csv')
wine = pd.concat([
    pd.read_csv('data/winequality-white.csv', sep=';').assign(kind='white'), 
    red_wine.assign(kind='red')
])

## Train Test Split

Rather than having to write something like this every time:

```python
shuffled = planets.reindex(np.random.permutation(planets.index))
train_end_index = int(np.ceil(shuffled.shape[0] * .75))
training = shuffled.iloc[:train_end_index,]
testing = shuffled.iloc[train_end_index:,]
```

We can use scikit-learn's `train_test_split()` function to get our training and testing sets. (We will discuss the validation set in chapter 10.)

In [ ]:
from sklearn.model_selection import train_test_split

X = planets[['eccentricity', 'semimajoraxis', 'mass']]
y = planets.period

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=0
)

The original data had this shape:

In [ ]:
X.shape, y.shape

Our training data has this shape:

In [ ]:
X_train.shape, y_train.shape

Our testing data has this shape:

In [ ]:
X_test.shape, y_test.shape

Let's look at the first 5 entries:

In [ ]:
X_train.head()

Our y data will be for the same rows as our X:

In [ ]:
y_train.head()

## Scaling data
### Standardizing with `StandardScaler`

In [ ]:
from sklearn.preprocessing import StandardScaler

standardized = StandardScaler().fit_transform(X_train)

# examine some of the non-NaN values
standardized[~np.isnan(standardized)][:30]

### Normalizing with `MinMaxScaler`

In [ ]:
from sklearn.preprocessing import MinMaxScaler

normalized = MinMaxScaler().fit_transform(X_train)

# examine some of the non-NaN values
normalized[~np.isnan(normalized)][:30]

### Using the Median and IQR with `RobustScaler`

In [ ]:
from sklearn.preprocessing import RobustScaler

robust_scaled = RobustScaler().fit_transform(X_train)

# examine some of the non-NaN values
robust_scaled[~np.isnan(robust_scaled)][:30]

## Encoding
### Binary encoding with `np.where()`

In [ ]:
np.where(wine.kind == 'red', 1, 0)

We can also use the `LabelBinarizer` class from scikit-learn. By calling the `inverse_transform()` method, we see the labels assigned to each value:

In [ ]:
from sklearn.preprocessing import LabelBinarizer

binary_labels = LabelBinarizer().fit(wine.kind)
binary_labels.inverse_transform(np.array([0, 1]))

We can use the `Binarizer` class for binary encoding of values based on a threshold. Values less than or equal to `threshold` will be 0; values greater than `threshold` will be 1:

In [ ]:
from sklearn.preprocessing import Binarizer

pd.Series(
    Binarizer(threshold=6).fit_transform(red_wine.quality.values.reshape(-1, 1)).flatten()
).value_counts()

### Ordinal Encoding with `LabelEncoder`

In [ ]:
from sklearn.preprocessing import LabelEncoder

pd.Series(LabelEncoder().fit_transform(pd.cut(
    red_wine.quality,
    bins=[-1, 3, 6, 10],
    labels=['0-3 (low)', '4-6 (med)', '7-10 (high)']
))).value_counts()

### One-hot encoding
In some cases, label encoding may yield some associations that aren't something we want the model to be trained on. A safer strategy is to use one-hot encoding.

Our planets data has a `list` column that we can one-hot encode:

In [ ]:
planets.list.value_counts()

We can use `pd.get_dummies()` to one-hot encode this information:

In [ ]:
pd.get_dummies(planets.list).head()

This gives us a redundant column. Note that we only need one less column than the number of planet lists. Pandas makes it easy to remove one of the columns to address multicollinearity:

In [ ]:
pd.get_dummies(planets.list, drop_first=True).head()

We can also use the `LabelBinarizer` class:

In [ ]:
from sklearn.preprocessing import LabelBinarizer

LabelBinarizer().fit_transform(planets.list)

## Imputing
The planets data has some missing values. We can use imputing strategies to avoid having to drop them from our model.

In [ ]:
planets[['semimajoraxis', 'mass', 'eccentricity']].tail()

### `SimpleImputer`
We can fill with the `mean`, `median`, `most_frequent` (mode), or a constant value by specifiying the `strategy`. The default is the mean:

In [ ]:
from sklearn.impute import SimpleImputer

SimpleImputer().fit_transform(
    planets[['semimajoraxis', 'mass', 'eccentricity']]
)

Changing to the median is just a matter of passing that as the `strategy`:

In [ ]:
from sklearn.impute import SimpleImputer

SimpleImputer(strategy='median').fit_transform(
    planets[['semimajoraxis', 'mass', 'eccentricity']]
)

### `KNNImputer`
Since this data isn't something that is easily measured, assuming that the planets we don't have the data for are similar to the rest is dangerous. It could be that the ones that have missing data have something in common. Replacing missing values for the semi-major axis with the average of the ones we know is hardly a good strategy. Instead, we could try to use the mass and eccentricity columns to find similar planets and use their semi-major axes to impute the missing data. This can be done with the `KNNImputer` class. Notice the first column in the bottom 3 rows (the imputed semi-major axis is drastically different from what we got using the overall mean:

In [ ]:
from sklearn.impute import KNNImputer

KNNImputer().fit_transform(
    planets[['semimajoraxis', 'mass', 'eccentricity']]
)

### `MissingIndicator`
In some cases, we don't want to fill in a value, but rather use the fact that the data is missing as a feature in our model:

In [ ]:
from sklearn.impute import MissingIndicator

MissingIndicator().fit_transform(
    planets[['semimajoraxis', 'mass', 'eccentricity']]
)

## Additional Transformers
### `FunctionTransformer`
With the `FunctionTransformer` class, we can use any function on the data. By passing `validate=True`, we will convert the result to two-dimensional NumPy array and raise an error if there is an issue:

In [ ]:
from sklearn.preprocessing import FunctionTransformer

FunctionTransformer(
    np.abs, validate=True
).fit_transform(X_train.dropna())

### `ColumnTransformer`
Sometimes we don't want to perform the same transformation on all of our features, the `ColumnTransformer` class lets us specify which tranformations to use on each column. We pass a list of tuples in the form `(name, transformer object, columns to apply to)`:

In [ ]:
from sklearn.compose import ColumnTransformer 
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler, StandardScaler

ColumnTransformer([
    ('impute', KNNImputer(), [0]),
    ('standard_scale', StandardScaler(), [1]),
    ('min_max', MinMaxScaler(), [2])
]).fit_transform(X_train)[10:15]

We can also use the `make_column_transformer()` function, which will name the transformers for us:

In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

categorical = [
    col for col in planets.columns
    if col in [
        'list', 'name', 'description', 
        'discoverymethod', 'lastupdate'
    ]
]
numeric = [col for col in planets.columns if col not in categorical]

make_column_transformer(
    (StandardScaler(), numeric),
    (OneHotEncoder(sparse=False), categorical)
).fit_transform(planets.dropna())

## `Pipeline`
Using pipelines ensures the whole model training and testing process is consistent. To make a pipeline, we pass in a list of steps as tuples of `(name, object)`:

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

Pipeline([('scale', StandardScaler()), ('lr', LinearRegression())])

We aren't limited to using pipelines with models &mdash; they can be used inside other `sklearn` objects. This makes it possible for us to first use k-NN imputing on the semi-major axis data and then standard scale the result:

In [ ]:
from sklearn.compose import ColumnTransformer 
from sklearn.impute import KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler

ColumnTransformer([
    ('impute', Pipeline([
        ('impute', KNNImputer()), ('scale', StandardScaler())
    ]), [0]),
    ('standard_scale', StandardScaler(), [1]),
    ('min_max', MinMaxScaler(), [2])
]).fit_transform(X_train)[10:15]

We can then include this as part of a pipeline, which gives us tremendous flexibility in how we build our models:

In [ ]:
Pipeline([
    (
        'preprocessing', 
        ColumnTransformer([
            ('impute', Pipeline([
                ('impute', KNNImputer()), ('scale', StandardScaler())
            ]), [0]),
            ('standard_scale', StandardScaler(), [1]),
            ('min_max', MinMaxScaler(), [2])
        ])
    ),
    ('model', LinearRegression())
])

We can also use the `make_pipeline()` function to make the pipeline without naming the steps ourselves:

In [ ]:
from sklearn.pipeline import make_pipeline

make_pipeline(StandardScaler(), LinearRegression())

<hr>
<div style="overflow: hidden; margin-bottom: 10px;">
    <div style="float: left;">
        <a href="./planets_ml.ipynb">
            <button>Planets</button>
        </a>
        <a href="./red_wine.ipynb">
            <button>Red Wine</button>
        </a>
        <a href="./wine.ipynb">
            <button>Red + White Wine</button>
        </a>
    </div>
    <div style="float: right;">
        <a href="../ch_09/planets_ml.ipynb">
            <button>Next Notebook &#8594;</button>
        </a>
    </div>
</div>
<hr>